In [1]:
!pip install -U py-boost

  Obtaining dependency information for py-boost from https://files.pythonhosted.org/packages/35/fe/8ffba87d7701b9952e44d3e275aa682a3bbdb5179381516082b91828fc85/py_boost-0.4.3-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 889.7 kB/s eta 0:00:00--:--


In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import cupy as cp
from py_boost.gpu.utils import *

### Введение

Кернел оперирует со следующими сущностями:

1) Квантизованая матрица признаков **X** размера (n_rows, n_columns) типа uint8. Диапазон значений в матрице [0; **max_bin**), **max_bin** <= 256

2) Матрица таргетов **Y** размера (n_rows, n_out) типа float32

3) Вектор индекса текущей ноды **nodes** дерева размера (n_rows, ) типа int32, принимает значения [0, **nnodes**), где **nnodes** в теории не ограничено, но на практике от 1 до 256, больше редко, но может быть и 1000 и больше


Поскольку иногда расчет надо проводить по разным подвыборкам строк/колонок/выходов, нам нужны вспомогательные указатели типа uint64:

1) row_indexer

2) col_indexer

3) out_indexer



Кернел для подсчета гистограмм в одном треде обрабатывает 4 фичи за раз. Это происходит таким образом:

1) Нужно западить размер фичей, чтобы делилось на 4, например, если у нас 10 фичей, создаем сбоку 2 пустые колонки, итого получается 12

2) исходный тип фичи - uint8, теперь мы можем сгруппировать 4ки фичей, представив как массив uint32

За эту операцию отвечает функция pad_and_move, она принимает массив на CPU, делает паддинг и копирует на GPU

### Сгенерируем случайные вводные для кернела 

In [13]:
%%time
# параметры, с которыми тестируем
n_rows = 1000000
n_cols = 99
n_out = 10
max_bin = 256
nnodes = 32

# параметры семплирования

colsample = .8
subsample = .8
outsample = 1. # в реале outsample всегда будет 1

# генерируем данные
X_cpu = np.random.randint(0, max_bin, size=(n_rows, n_cols)).astype(np.uint8)
print('Initial CPU features shape', X_cpu.shape)

X = pad_and_move(X_cpu)
print('Padded GPU features shape', X.shape)

nodes = cp.random.randint(0, nnodes, size=(n_rows, )).astype(np.int32)
print('Nodes GPU vector shape', nodes.shape)

Y = cp.random.rand(n_rows, n_out).astype(np.float32)
print('Targets GPU array shape', Y.shape)

# генерируем указатели, по которым будем считать гистограмму 
def sample_idx(n, sample):
    
    idx = cp.arange(n, dtype=cp.uint64)
    sl = cp.random.rand(n) < sample
    
    return cp.ascontiguousarray(idx[sl])


row_indexer = sample_idx(n_rows, subsample)
print('Sampled rows', row_indexer.shape)

col_indexer = sample_idx(n_cols, colsample)
print('Sampled cols', col_indexer.shape)

out_indexer = sample_idx(n_out, outsample)
print('Sampled out', out_indexer.shape)

Initial CPU features shape (1000000, 99)
Padded GPU features shape (1000000, 99)
Nodes GPU vector shape (1000000,)
Targets GPU array shape (1000000, 10)
Sampled rows (799883,)
Sampled cols (77,)
Sampled out (10,)
CPU times: user 480 ms, sys: 220 ms, total: 701 ms
Wall time: 699 ms


Здесь стоит пояснить. Как говорилось выше, матрица фичей падится так, чтобы X.shape[1] делилось на 4. Но как мы видим, шейпы не изменились. Это потому что на самом деле X не contiguous массив, то есть внутри он разрывный. Как это сделано в pad_and_move:

1) аллоцируется массив X (n_rows, 100)

2) X[:, :99] заполняется X_cpu

3) возвращается X[:, :99]

То есть шейп у него n_rows, 99, но уложен он внутри так, как будто n_rows, 100. Это важно понимать, чтобы разобраться в текущем кернеле

Я не настаиваю, чтобы в новых кернелах эта логика сохранялась. Я делал это только с целью максимально ускорить существующие кернелы. Но вообще, похожий трюк с 4ками используют более менее все, я его не придумал

# Продолжим пулять кернел

In [14]:
%%time
# считаем фактическое количество аутпутов/колонок, по которым считаем гистограмму
# так как у нас есть семплирование, оно может отличаться от размеров датасета
nout = out_indexer.shape[0]
nfeats = col_indexer.shape[0]

CPU times: user 2.19 ms, sys: 0 ns, total: 2.19 ms
Wall time: 1.18 ms


ну и теперь все готово, осталось самое интересное - запустить подсчет

In [35]:
%%timeit
# аллоцируем гистограмму
res = cp.zeros((nout, nnodes, nfeats, max_bin), dtype=cp.float32)
# заполним гистограмму
fill_histogram(
    res, X, Y, nodes, 
    col_indexer, row_indexer, out_indexer
)
cp.cuda.Device(0).synchronize()

48.5 ms ± 1.33 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [36]:
res.sum()

array(3.078966e+08, dtype=float32)

Для проверки - должно получаться nfeats * Y[row_indexer].sum()

In [38]:
nfeats * Y[row_indexer].sum()

array(3.0789658e+08, dtype=float32)